<a href="https://colab.research.google.com/github/cybertraining-dsc/sp21-599-354/blob/main/projects/code/CNN_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cloudmesh-common

# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

from cloudmesh.common.StopWatch import StopWatch


In [2]:

# loading dataset
train = pd.read_csv('/content/drive/MyDrive/plant_data/train_str.csv')
#train = pd.read_csv('/content/drive/MyDrive/plant_data/train_rot.csv')
test = pd.read_csv('/content/drive/MyDrive/plant_data/test.csv')
predict = pd.read_csv('/content/drive/MyDrive/plant_data/test.csv')


train.head()

,id,label
0,205,1
1,6,0
2,202,1
3,148,1
4,383,2


In [3]:
# loading training images
train_img = []

for img_name in tqdm(train['id']):
    # defining the image path
    image_path = f'/content/drive/MyDrive/plant_data/train_str/{img_name}.png'
    # reading the image
    img = imread(image_path, as_gray=True)
    # normalizing the pixel values
    img /= 255.0
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    train_img.append(img)

# converting the list to numpy array
train_x = np.array(train_img)
# defining the target
train_y = train['label'].values
train_x.shape

100%|██████████| 420/420 [00:02<00:00, 165.06it/s]


(420, 400, 400)

(<matplotlib.axes._subplots.AxesSubplot at 0x7f03c01ebd90>,
 <matplotlib.image.AxesImage at 0x7f03c016e250>)

In [4]:
# visualizing images
i = 0
plt.figure(figsize=(10,10))
plt.subplot(221), plt.imshow(train_x[i], cmap='gray')
plt.subplot(222), plt.imshow(train_x[i+25], cmap='gray')
plt.subplot(223), plt.imshow(train_x[i+50], cmap='gray')
plt.subplot(224), plt.imshow(train_x[i+75], cmap='gray')

In [5]:
# create validation set
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((378, 400, 400), (378,)), ((42, 400, 400), (42,)))

In [6]:
# converting training images into torch format
train_x = train_x.reshape(378, 1, 400, 400)
#train_x = train_x.reshape(1512, 1, 400, 400)

train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)

# shape of training data
train_x.shape, train_y.shape

(torch.Size([378, 1, 400, 400]), torch.Size([378]))

In [7]:
# converting validation images into torch format
val_x = val_x.reshape(42, 1, 400, 400)
#val_x = val_x.reshape(168, 1, 400, 400)
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int);
val_y = torch.from_numpy(val_y)

# shape of validation data
val_x.shape, val_y.shape

(torch.Size([42, 1, 400, 400]), torch.Size([42]))

In [8]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(40000, 10)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [9]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=40000, out_features=10, bias=True)
  )
)


In [10]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x), Variable(train_y)
    # getting the validation set
    x_val, y_val = Variable(val_x), Variable(val_y)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [11]:
StopWatch.start("Train")
# defining the number of epochs
n_epochs = 15
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
    train(epoch)

StopWatch.stop("Train")

Epoch :  1 	 loss : tensor(2.4620, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  3 	 loss : tensor(259.3098, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  5 	 loss : tensor(37.9111, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  7 	 loss : tensor(89.5009, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  9 	 loss : tensor(56.7021, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  11 	 loss : tensor(43.8870, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  13 	 loss : tensor(26.5657, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  15 	 loss : tensor(16.5749, device='cuda:0', grad_fn=<NllLossBackward>)


In [12]:
# plotting the training and validation loss
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.show()

In [13]:
# prediction for training set
with torch.no_grad():
    output = model(train_x.cuda())
    
softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on training set
accuracy_score(train_y, predictions)

0.6402116402116402

In [14]:

# prediction for validation set
with torch.no_grad():
    output = model(val_x.cuda())

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on validation set
accuracy_score(val_y, predictions)

0.5476190476190477

In [15]:
# loading test images
test_img = []
for img_name in tqdm(test['id']):
    # defining the image path
    image_path = f'/content/drive/MyDrive/plant_data/test/{img_name}.png'
    # reading the image
    img = imread(image_path, as_gray=True)
    # normalizing the pixel values
    img /= 255.0
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    test_img.append(img)

# converting the list to numpy array
test_x = np.array(test_img)
test_x.shape

100%|██████████| 47/47 [00:00<00:00, 163.43it/s]


(47, 400, 400)

In [16]:
# converting training images into torch format
test_x = test_x.reshape(47, 1, 400, 400)
test_x  = torch.from_numpy(test_x)
test_x.shape

torch.Size([47, 1, 400, 400])

In [17]:
StopWatch.start("Test")
# generating predictions for test set
with torch.no_grad():
    output = model(test_x.cuda())

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)
StopWatch.start("Test")

In [18]:
# replacing the label with prediction
predict['label'] = predictions
predict.head()

,id,label
0,400,1
1,300,1
2,10,1
3,250,1
4,40,2


In [19]:
# saving the file
predict.to_csv('/content/drive/MyDrive/plant_data/str_predict.csv', index=False)
#predict.to_csv('/content/drive/MyDrive/plant_data/rot_predict.csv', index=False)

In [20]:
StopWatch.benchmark()


+---------------------+------------------------------------------------------------------+
| Attribute           | Value                                                            |
|---------------------+------------------------------------------------------------------|
| BUG_REPORT_URL      | "https://bugs.launchpad.net/ubuntu/"                             |
| DISTRIB_CODENAME    | bionic                                                           |
| DISTRIB_DESCRIPTION | "Ubuntu 18.04.5 LTS"                                             |
| DISTRIB_ID          | Ubuntu                                                           |
| DISTRIB_RELEASE     | 18.04                                                            |
| HOME_URL            | "https://www.ubuntu.com/"                                        |
| ID                  | ubuntu                                                           |
| ID_LIKE             | debian                                                           